In [1]:
import numpy as np
import json
import requests
import pandas as pd
import datetime
import dash_table
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output
from plotly.graph_objs import *
from scipy.integrate import odeint
from scipy.optimize import curve_fit


In [ ]:
def data():
    data_confirmed = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
    data_confirmed = data_confirmed.drop(["Province/State","Lat","Long"],axis=1)
    data_confirmed = data_confirmed.groupby(["Country/Region"]).sum()
    
    recovered_data=pd.read_csv("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv")
    recovered_data=recovered_data.drop(["Province/State","Lat","Long"],axis=1)
    recovered_data = recovered_data.groupby(["Country/Region"]).sum()
    
    deaths_data=pd.read_csv("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
    deaths_data=deaths_data.drop(["Province/State","Lat","Long"],axis=1)
    deaths_dta = deaths_data.groupby(["Country/Region"]).sum()
    
    return data_confirmed, recovered_data, deaths_data

def data_table():
    
    data_confirmed, recovered_data, deaths_data = data()
    
    last_date = deaths_data.T.index[-1]
    data_table = pd.DataFrame([data_confirmed[last_data], recovered_data[last_data], deaths_data[last_date]]).T
    data_table.columns = ["confirmed", "recovered", "deaths"]
    data_table["active"] = data_table.confirmed - data_table.recovered - data_table.deaths
    data_table ["percent_deaths"] = np.round(data_table.deaths / data_table.confirmed, 2)
    data_table ["percent_recoveries"] = np.round(data_table.recovered / data_table.confirmed, 2)
    data_table = data_table.sort_values(by = 'confirmed', ascending = False)
    data_table.reset_index(level=0, inplace=True)
    
    return data_table
def sir_simulations( confirmed_data, recovered_data, dates):
    
    duration_for_simulation = 30 # duration for simulation
    
    confirmed_data = confirmed_data[(len(confirmed_data)- duration_for_simulation):]
    
    recovered_data = recovered_data[(len(recovered_data)- duration_for_simulation):]
    
    dates = dates[ len(dates)-duration_for_simulation: ]
    N = 1000000
    I_0 = confirmed_data[0]
    R_0 = recovered_data[0]
    S_0 = N - R_0 - I_0
    
    
    def SIR(y, t, beta, gamma):
        S = y[0]
        I = y[1]
        R = y[2]
        return -beta*SI/N, (beta*S*I)/N-(gamma*I), gamma*I
    
    def fit_odeint(t,beta, gamma):
        return odeint(SIR,(S_0,I_0,R_0), t, args = (beta,gamma))[:,1]
    
    t = np.arrange(len(confirmed_data))
    params, cerr = curve_fir(fit_odeint,t, confirmed_data)
    prediction = list(fit_odeint(t,beta,gamma))
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x= dates, y= prediction,
                        mode='linemarkers',
                        name='Simulated'))
    fig.add_bar(x= data, y= confirmed_data, name = "Actual")
    fig.update_layout(height = 800,
    title={
        'text' :"SIR simulations",
        'x' : 0.5},
    xaxis_title="Date",
    yaxis_title="Infections")                  
    
    



























